In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stanford-question-answering-dataset/train-v1.1.json
/kaggle/input/stanford-question-answering-dataset/dev-v1.1.json


In [2]:
import json
import torch
from datasets import Dataset, DatasetDict
from transformers import  AlbertTokenizerFast, AlbertForQuestionAnswering, TrainingArguments, Trainer

2024-07-21 04:51:40.189134: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-21 04:51:40.189252: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-21 04:51:40.330051: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
torch.cuda.is_available()

True

In [4]:
!nvidia-smi

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Sun Jul 21 04:52:28 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P0             27W /  250W |       3MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
def squad_json_to_dataset(squad_dict):
    contexts = []
    questions = []
    answers = []

    for group in squad_dict['data']:
        for paragraph in group['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                if 'answers' in qa:
                    for answer in qa['answers']:
                        contexts.append(context)
                        questions.append(question)
                        answers.append(answer)
                else:
                    contexts.append(context)
                    questions.append(question)
                    answers.append({'answer_start': -1, 'text': ''})

    return Dataset.from_dict({'context': contexts, 'question': questions, 'answers': answers})

In [8]:
with open('/kaggle/input/stanford-question-answering-dataset/train-v1.1.json', 'r') as f:
    squad_train = json.load(f)

with open('/kaggle/input/stanford-question-answering-dataset/dev-v1.1.json', 'r') as f:
    squad_dev = json.load(f)

In [9]:
train_dataset = squad_json_to_dataset(squad_train)
dev_dataset = squad_json_to_dataset(squad_dev)

In [10]:
datasets = DatasetDict({'train': train_dataset, 'validation': dev_dataset})

In [11]:
tokenizer = AlbertTokenizerFast.from_pretrained('albert-base-v2')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

In [12]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples['question']]
    contexts = [c.strip() for c in examples['context']]
    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]

    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        if answer['answer_start'] == -1:
            start_positions.append(0)
            end_positions.append(0)
        else:
            start_char = answer["answer_start"]
            end_char = start_char + len(answer["text"])

            sequence_ids = inputs.sequence_ids(i)

            context_start = sequence_ids.index(1)
            context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

            # Find the start and end token positions within the context
            token_start_index = 0
            while token_start_index < len(offset) and offset[token_start_index][0] <= start_char:
                token_start_index += 1
            token_end_index = token_start_index
            while token_end_index < len(offset) and offset[token_end_index][1] <= end_char:
                token_end_index += 1

            if token_start_index >= len(offset) or token_end_index >= len(offset) or sequence_ids[token_start_index] != 1 or sequence_ids[token_end_index - 1] != 1:
                start_positions.append(0)
                end_positions.append(0)
            else:
                start_positions.append(token_start_index - 1)
                end_positions.append(token_end_index - 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [13]:
tokenized_dataset = datasets.map(preprocess_function, batched=True, remove_columns=datasets["train"].column_names)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/34726 [00:00<?, ? examples/s]

In [14]:
model = AlbertForQuestionAnswering.from_pretrained('albert-base-v2')

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForQuestionAnswering were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [27]:
os.environ['WANDB_DISABLED'] = 'true'

In [28]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.874600,0.992390
2,0.711900,1.019374
3,0.513300,1.113644
4,0.380400,1.316826
5,0.252300,1.532058


TrainOutput(global_step=27375, training_loss=0.5691485788005672, metrics={'train_runtime': 20613.5195, 'train_samples_per_second': 21.248, 'train_steps_per_second': 1.328, 'total_flos': 7254446515914240.0, 'train_loss': 0.5691485788005672, 'epoch': 5.0})

In [29]:
eval_results = trainer.evaluate()
print(eval_results)

model.save_pretrained('model')
tokenizer.save_pretrained('tokenizer')

{'eval_loss': 1.532057762145996, 'eval_runtime': 529.6603, 'eval_samples_per_second': 65.563, 'eval_steps_per_second': 4.099, 'epoch': 5.0}


('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [30]:
# Load the tokenizer and model from the saved directory
tokenizer = AlbertTokenizerFast.from_pretrained('tokenizer')
model = AlbertForQuestionAnswering.from_pretrained('model')

# Function to get a prediction
def get_prediction(context, question):
    # Tokenize the inputs
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt', truncation=True)
    
    # Get the model's output
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract start and end logits
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Get the most likely beginning and end of the answer
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits) + 1

    # Decode the tokens to get the answer
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs.input_ids[0][start_index:end_index]))

    return answer

# Sample paragraph and question
context = "The Apollo program was the third United States human spaceflight program carried out by the National Aeronautics and Space Administration (NASA), which succeeded in landing the first humans on the Moon from 1969 to 1972. The first manned flight of Apollo was in 1968, and it achieved the first manned moon landing on July 20, 1969, with Apollo 11."
question = "When did the first manned moon landing occur?"

# Get the prediction
answer = get_prediction(context, question)
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: When did the first manned moon landing occur?
Answer: july 20, 1969


In [31]:
context = "The Amazon Rainforest, also known as Amazonia, is a moist broadleaf tropical rainforest in the Amazon biome that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 km2 (2,700,000 sq mi), of which 5,500,000 km2 (2,100,000 sq mi) are covered by the rainforest. This region includes territory belonging to nine nations. The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, Colombia with 10%, and with minor amounts in Venezuela, Ecuador, Bolivia, Guyana, Suriname, and French Guiana."
question = "Which country contains the majority of the Amazon Rainforest?"

# Get the prediction
answer = get_prediction(context, question)
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: Which country contains the majority of the Amazon Rainforest?
Answer: brazil


In [32]:
context = "The Great Wall of China is a series of fortifications that were built across the historical northern borders of ancient Chinese states and Imperial China as protection against various nomadic groups from the Eurasian Steppe. Several walls were built from as early as the 7th century BC, with selective stretches later joined by Qin Shi Huang, the first Emperor of China. Little of the Qin wall remains. Later on, many successive dynasties have built and maintained multiple stretches of border walls. The most well-known sections of the wall were built by the Ming dynasty (1368–1644)."
question = "Which dynasty built the most well-known sections of the Great Wall of China?"

# Get the prediction
answer = get_prediction(context, question)
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: Which dynasty built the most well-known sections of the Great Wall of China?
Answer: the ming dynasty


In [33]:
context = "The Industrial Revolution, which took place from the 18th to 19th centuries, was a period during which predominantly agrarian, rural societies in Europe and America became industrial and urban. Prior to the Industrial Revolution, which began in Britain in the late 1700s, manufacturing was often done in people’s homes, using hand tools or basic machines. Industrialization marked a shift to powered, special-purpose machinery, factories, and mass production. The iron and textile industries, along with the development of the steam engine, played central roles in the Industrial Revolution, which also saw improved systems of transportation, communication, and banking. While industrialization brought about an increased volume and variety of manufactured goods and an improved standard of living for some, it also resulted in often-grim employment and living conditions for the poor and working classes. To explore the effects of the Industrial Revolution on the social, economic, and cultural conditions of the times, one must look at the profound changes it wrought in every aspect of society. Cities and towns expanded rapidly as people moved from rural areas to work in factories. This led to significant urbanization, which had both positive and negative consequences. On the positive side, the rise of factories provided job opportunities, leading to a boom in employment and the growth of a working middle class. Economies of scale meant that goods could be produced in greater quantities and at lower prices. This contributed to increased availability and affordability of goods, which improved the overall quality of life for many people. The wealth generated by industrial growth also led to advancements in infrastructure, such as roads, bridges, and railways, which further fueled economic development and societal progress. However, the Industrial Revolution also had several negative effects. The rapid growth of factories and urban areas resulted in overcrowded and unsanitary living conditions. Many workers lived in cramped, poorly built housing with inadequate ventilation and sanitation facilities. Working conditions in factories were often harsh and dangerous, with long hours, low wages, and little regard for worker safety. Child labor was rampant, and children as young as five years old were employed in factories under deplorable conditions. Environmental degradation was another significant consequence of industrialization. The massive increase in coal and other fossil fuel consumption led to widespread air and water pollution. Industrial waste was often dumped into rivers and lakes, causing severe contamination and health hazards. The shift from agrarian to industrial societies also disrupted traditional ways of life and had a profound impact on family structures and community dynamics. Despite these challenges, the Industrial Revolution was a catalyst for innovation and technological progress. It paved the way for the modern era, bringing about significant advancements in science, engineering, and medicine. The period also saw the rise of labor unions and movements advocating for workers' rights, which eventually led to improved working conditions and labor laws. The legacy of the Industrial Revolution is complex, as it brought both tremendous benefits and profound social challenges. Its impact continues to be felt in the modern world, shaping contemporary economic and social landscapes."
question1 = "What were some of the positive effects of the Industrial Revolution on society?"
question2 = "What were some of the negative consequences of the Industrial Revolution, particularly for the working class?"


answer1 = get_prediction(context, question1)
print(f"Question: {question1}")
print(f"Answer: {answer1}")

answer2 = get_prediction(context, question2)
print(f"Question: {question2}")
print(f"Answer: {answer2}")

Question: What were some of the positive effects of the Industrial Revolution on society?
Answer: the rise of factories provided job opportunities, leading to a boom in employment and the growth of a working middle class
Question: What were some of the negative consequences of the Industrial Revolution, particularly for the working class?
Answer: overcrowded and unsanitary living conditions
